# BITS F312 - Neural Network and Fuzzy Logic



# NNFL Assignment 2

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Changing directory to the directory containing dataset
%cd drive/MyDrive/NNFL/Data_A2/

/content/drive/MyDrive/NNFL/Data_A2


In [ ]:
# listing datasets
%ls -l

total 1234719
-rw------- 1 root root     637638 Oct 31 04:55 Assignment2.pdf
-rw------- 1 root root        259 Oct 31 04:57 class_label.mat
-rw------- 1 root root      40295 Oct 31 04:57 data55.xlsx
-rw------- 1 root root      21269 Oct 31 04:55 data5.xlsx
-rw------- 1 root root 1263647365 Oct 31 04:58 input.mat
drwx------ 2 root root       4096 Nov 18 05:58 logs/


In [ ]:
# libraries required
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from pprint import pprint

In [ ]:
# supressing warnings
import warnings
warnings.filterwarnings('ignore')

# Q3

The dataset (data5.xlsx) contains 7 features and the last column is the output (class labels). Design a
multilayer perceptron based neural network (two hidden layers) for the classification. You can use both
holdout (70, 10, and 20%) and 5-fold cross-validation approaches for evaluating the performance of the
classifier (individual accuracy and overall accuracy). You can select the number of hidden neurons of each
hidden layer and other MLP parameters using grid-search method. (Packages such as Scikitlearn, keras,
tensorflow, pytorch etc. are not allowed). 

In [ ]:
def relu(x):
  
  return np.maximum(x,0)

def reluDerivative(x):
  
    return np.heaviside(x,1)

def sigmoid(x):
  
  return 1/(1+ np.exp(-x))

def sigmoidDerivative(x):
  
    return x * (1 - x)


In [ ]:
def perceptronTraining(train_X, train_y, neuron_L1 = 32, neuron_L2 = 16, epochs = 5000, alpha = 0.00125):
  neuron_in = 147
  neuron_out = 1
  weight_L1 = np.random.uniform(size=(neuron_in,neuron_L1))
  bias_L1 = np.random.uniform(size=(1,neuron_L1))
  weight_L2 = np.random.uniform(size=(neuron_L1,neuron_L2))
  bias_L2 = np.random.uniform(size=(1,neuron_L2))
  final_weight = np.random.uniform(size=(neuron_L2,neuron_out))
  final_bias = np.random.uniform(size=(1,neuron_out))
  costs_perceptron = []
  for i in range(epochs):


    hiddenInpL1 = np.dot(train_X,weight_L1)
    hiddenInpL1=hiddenInpL1 + bias_L1
    hiddenActL1 = relu(hiddenInpL1)
    hiddenInpL2 = np.dot(hiddenActL1,weight_L2)
    hiddenInpL2 = hiddenInpL2 + bias_L2
    hiddenActL2 = relu(hiddenInpL2)
    outInpL = np.dot(hiddenActL2,final_weight)
    outInpL = outInpL+ final_bias
    output = sigmoid(outInpL)

    error_track = output-train_y
    slopeOutL = sigmoidDerivative(output)
    slopeHidL2 = reluDerivative(hiddenActL2)
    slopeHidL1 = reluDerivative(hiddenActL1)
    output_derivative = error_track * slopeOutL
    errorHidL2 = output_derivative.dot(final_weight.T)
    d_hidden2 = errorHidL2*slopeHidL2 
    errorHidL1 = d_hidden2.dot(weight_L2.T)
    d_hidden1 = errorHidL1*slopeHidL1
    final_weight = final_weight- hiddenActL2.T.dot(output_derivative) *alpha
    final_bias = final_bias- np.sum(output_derivative, axis=0,keepdims=True) *alpha
    weight_L2 = weight_L2- hiddenActL1.T.dot(d_hidden2) *alpha
    bias_L2 = bias_L2- np.sum(d_hidden2, axis=0,keepdims=True) *alpha
    weight_L1 = weight_L1- train_X.T.dot(d_hidden1) *alpha
    bias_L1 = bias_L1- np.sum(d_hidden1, axis=0,keepdims=True) *alpha

    error_track = np.asarray(error_track)
    costs_perceptron.append(np.sum(0.5*(error_track*error_track)))

  return weight_L1, weight_L2, final_weight, bias_L1, bias_L2, final_bias, costs_perceptron
def pred_eval(X, weight_L1, weight_L2, final_weight, bias_L1, bias_L2, final_bias):

    hiddenInpL1 = np.dot(X, weight_L1)
    hiddenInpL1 = hiddenInpL1 + bias_L1
    hiddenActL1 =  relu(hiddenInpL1)
    hiddenInpL2 = np.dot(hiddenActL1,weight_L2)
    hiddenInpL2 = hiddenInpL2 + bias_L2
    hiddenActL2 = relu(hiddenInpL2)
    outInpL = np.dot(hiddenActL2,final_weight)
    outInpL = outInpL+ final_bias
    output = sigmoid(outInpL)

    return output



In [ ]:
def resultQ3(filename = 'data5.xlsx'):
  dataset = pd.read_excel(filename, header = None)

  row, col = dataset.shape
  feats = col - 1 

  # normalization
  dataset.loc[:, dataset.columns != feats] = (dataset.loc[:, dataset.columns != feats]-dataset.loc[:, dataset.columns != feats].mean(axis=0))/dataset.loc[:, dataset.columns != feats].std(axis=0)
  
  # spliting dataset into train test and val
  training_data, validation_data, testing_data = np.split(dataset.sample(frac=1),[int(0.7*len(dataset)), int(0.8*len(dataset))])

  training_data = np.array(training_data)
  validation_data = np.array(validation_data)
  testing_data = np.array(testing_data)
  training_data_X = training_data[:, :feats]
  training_data_y = training_data[:, feats]
  validation_data_X = validation_data[:, :feats]
  validation_data_y = validation_data[:, feats]
  testing_data_X = testing_data[:, :feats]
  testing_data_y = testing_data[:, feats]

  train_row, train_col = training_data_X.shape

  weight_L1, weight_L2, final_weight, bias_L1, bias_L2, final_bias, costs_perceptron = perceptronTraining(training_data_X, training_data_y)
  temp_cost = costs_perceptron[-1]

  train_pred = pred_eval(training_data_X , weight_L1, weight_L2, final_weight, bias_L1, bias_L2, final_bias)
  train_pred = np.where(train_pred > 0.5, 1,0)
  print("Training Accuracy: \n")
  metrics(train_pred, training_data_y)
  print('{}'.format('-'*75))

  test_pred = pred_eval(testing_data_X , weight_L1, weight_L2, final_weight, bias_L1, bias_L2, final_bias)
  test_pred = np.where(test_pred > 0.5, 1,0)
  print("Testing Accuracy: \n")
  metrics(test_pred, testing_data_y)
  print('{}'.format('-'*75))
  
  validation_pred = pred_eval(validation_data_X , weight_L1, weight_L2, final_weight, bias_L1, bias_L2, final_bias)
  validation_pred = np.where(validation_pred > 0.5, 1,0)
  print("Validation Accuracy: \n")
  metrics(validation_pred, validation_data_y)
  print('{}'.format('-'*75))


In [ ]:
resultQ3()

Training Accuracy: 

Sensitivity :  0.8305084745762712
Specificity :  0.873015873015873
Accuracy ((TN+TP)/(TN+TP+FN+FP)) :  0.8524590163934426
---------------------------------------------------------------------------
Testing Accuracy: 

Sensitivity :  0.8706896551724138
Specificity :  0.8166666666666667
Accuracy ((TN+TP)/(TN+TP+FN+FP)) :  0.8432203389830508
---------------------------------------------------------------------------
Validation Accuracy: 

Sensitivity :  0.9285714285714286
Specificity :  0.8333333333333334
Accuracy ((TN+TP)/(TN+TP+FN+FP)) :  0.8793103448275862
---------------------------------------------------------------------------
